<H1>Stiff Skin Syndrome, Loeys et al. (2010)</H1>
<p>This notebook creates GA4GH Phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/20375004/" target="__blank">Loeys BL, et al., Mutations in fibrillin-1 cause congenital scleroderma: stiff skin syndrome. Sci Transl Med. 2010;2(23):23ra20. PMID:20375004</a>.</p>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.77


In [2]:
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_hpo(release='v2024-01-16')

In [3]:
PMID = "PMID:20375004"
title = "Mutations in fibrillin-1 cause congenital scleroderma: stiff skin syndrome"
cite = Citation(pmid=PMID, title=title)

metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo.version)
print(f"HPO version {hpo.version}")

HPO version 2024-01-16


In [4]:
from pyphetools.creation import HpoExactConceptRecognizer

hpo_cr = HpoExactConceptRecognizer.from_hpo(hpo)

In [5]:
df = pd.read_table('input/stiffskin_loeys2010.tsv')

In [6]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index
dft.head()

Identifier,Age(yrs),Sex,Height(cm),Stiff skin,Surgery,Ectopia Lentis,Cataract,Myopia,Arachnodactyly,Camptodactyly,Elbowcontractures,Limitedshoulderelevation,Kneecontractures,MCP/IPnodules,Aorta,Neuropathy,GER,Variant,patient_id
1-II:1,51,M,173,++,"Appendectomy, Herniorraphy, Coronaryarterybypass",─,─,+,─,++,++,─,++,++,nl,─,+,c.4710G>C,1-II:1
1-III:2,25,M,165,+++,Cesarean section,─,─,+,─,++,++,++,++,++,nl,+,+,c.4710G>C,1-III:2
2-III:1,70,M,163,+++,Gastric ulcer surgery,─*,+,─,─,++,++,++,++,++,nl,─,+,c.4710G>T,2-III:1
2-IV:2,37,F,155,+++,Cesarean Sections,─,─,─,─,++,++,++,++,++,nl,─,+,c.4710G>T,2-IV:2
3-I:1,54,F,145,+++,Hysterectomy,─,+,+,─,++,++,++,++,+,nl,─,─,c.4691G>C,3-I:1


<h2>Create mappers to convert data</h2>

In [7]:
column_mapper_list = list()

In [8]:
symbol_d = {'+': 'Stiff skin', '++': 'Stiff skin', '+++': 'Stiff skin'}
stiffSkinMapper = OptionColumnMapper(column_name='Stiff skin',concept_recognizer=hpo_cr, option_d=symbol_d)
stiffSkinMapper.preview_column(dft)
column_mapper_list.append(stiffSkinMapper)

In [9]:
items = {
  'Ectopia Lentis': ["Ectopia lentis","HP:0001083"],
  'Cataract': ['Cataract', 'HP:0000518'],
  'Myopia': ["Myopia", "HP:0000545"],
  'Arachnodactyly': ['Arachnodactyly', 'HP:0001166'],
  'Camptodactyly': ['Camptodactyly', 'HP:0012385'],
  'Elbowcontractures': ['Elbow contracture', 'HP:0034391'],
  'Limitedshoulderelevation': ['Limited shoulder flexion','HP:0033482'],
  'Neuropathy': ["Entrapment neuropathy",'HP:0012181'],
  'GER': ['Gastroesophageal reflux', 'HP:0002020']
}

item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, 
                                                            observed={'+', '++'},
                                                              excluded='-')

# Transfer to column_mapper_d
for k, v in item_column_mapper_d.items():
    column_mapper_list.append(v)

In [10]:
# Aorta
# Bicuspid aortic valve HP:0001647
mapper = SimpleColumnMapper(column_name='Aorta',
                            hpo_id='HP:0001647',
    hpo_label='Bicuspid aortic valve',
    observed='BAV',
    excluded='─')
column_mapper_list.append(mapper)
mapper.preview_column(dft)

,mapping,count
0,"original value: ""nl"" -> HP: Bicuspid aortic valve (HP:0001647) (not measured)",7
1,"original value: ""BAV"" -> HP: Bicuspid aortic valve (HP:0001647) (observed)",1


<h2>Demographic columns</h2>

In [11]:
ageMapper = AgeColumnMapper.by_year('Age(yrs)')
#ageMapper.preview_column(dft)

In [12]:
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
#sexMapper.preview_column(dft)

<h2>Variant data</H2>

In [13]:
fbn1_transcript='NM_000138.5' 
vman = VariantManager(df=dft, 
                      transcript=fbn1_transcript, 
                      gene_symbol="FBN1", 
                      allele_1_column_name='Variant', 
                      individual_column_name="patient_id")
varMapper = VariantColumnMapper(variant_d=vman.get_variant_d(),
                                variant_column_name='Variant', 
                                default_genotype='heterozygous')

In [14]:
pmid = "PMID:20375004"
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="patient_id", 
                        age_of_onset_mapper=ageMapper, 
                        sexmapper=sexMapper,
                        metadata=metadata,
                        variant_mapper=varMapper)
sss = Disease(disease_id='OMIM:184900', disease_label='Stiff skin syndrome')
encoder.set_disease(sss)
individuals = encoder.get_individuals()

In [15]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,42


In [16]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1-II:1 (MALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4710G>C (heterozygous),Stiff skin (HP:0030053); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Gastroesophageal reflux (HP:0002020)
1-III:2 (MALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4710G>C (heterozygous),Stiff skin (HP:0030053); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482); Entrapment neuropathy (HP:0012181); Gastroesophageal reflux (HP:0002020)
2-III:1 (MALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4710G>T (heterozygous),Stiff skin (HP:0030053); Cataract (HP:0000518); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482); Gastroesophageal reflux (HP:0002020)
2-IV:2 (FEMALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4710G>T (heterozygous),Stiff skin (HP:0030053); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482); Gastroesophageal reflux (HP:0002020)
3-I:1 (FEMALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4691G>C (heterozygous),Stiff skin (HP:0030053); Cataract (HP:0000518); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482)
3-II:2 (FEMALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4691G>C (heterozygous),Stiff skin (HP:0030053); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482); Bicuspid aortic valve (HP:0001647)
3-II:3 (FEMALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4691G>C (heterozygous),Stiff skin (HP:0030053); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482)
4-II:1 (MALE; n/a),Stiff skin syndrome (OMIM:184900),NM_000138.5:c.4729T>G (heterozygous),Stiff skin (HP:0030053); Myopia (HP:0000545); Camptodactyly (HP:0012385); Elbow contracture (HP:0034391); Limited shoulder flexion (HP:0033482); Entrapment neuropathy (HP:0012181)


In [17]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata,
                                              outdir=output_directory)

We output 8 GA4GH phenopackets to the directory phenopackets
